<a href="https://colab.research.google.com/github/KubX44/Tp-Spark-MLlib/blob/main/TP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Stocker le contenu de tous les fichiers du dossier tp4_data dans un DataFrame.

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=fd70446e66a57259145350ba43cf89556b823cc727a1aae47fcb2b4f32c77d12
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tp4").getOrCreate()

In [4]:
DataFrame = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/content/*.csv")

2) Afficher le schéma du Dataframe obtenu


In [5]:
DataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



3) Remplir les valeurs manquantes (NaN) avec la valeur 0.


In [6]:
DataFrame_filled = DataFrame.fillna(0)
DataFrame_filled.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6|2010-12-06 08:34:00|     2.95|   15987.0|United Kingdom|
|   537226|    21713|CITRONELLA CANDLE...|       8|2010-12-06 08:34:00|      2.1|   15987.0|United Kingdom|
|   537226|    22927|GREEN GIANT GARDE...|       2|2010-12-06 08:34:00|     5.95|   15987.0|United Kingdom|
|   537226|    20802|SMALL GLASS SUNDA...|       6|2010-12-06 08:34:00|     1.65|   15987.0|United Kingdom|
|   537226|    22052|VINTAGE CARAVAN G...|      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
|   537226|    22705|   WRAP GREEN PEARS |      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|
|   537226|    20781|GOLD EA

4) Ajouter une nouvelle colonne nommée "day_of_week". La valeur de la colonne est le
jour de la semaine correspondant à la date de chaque ligne dans la colonne
"InvoiceDate".

In [11]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = DataFrame\
.na.fill(0)\
.withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\





5) Diviser les données en un ensemble d'apprentissage et un ensemble de test. Effectuer
la division en se basant sur l'attribut InvoiceDate : l'ensemble d'apprentissage contient
les achats effectués avant 2010-12-13 et l'ensemble de test contient les achats effectués
durant ou après 2010-12-13.


In [12]:
trainDataFrame = preppedDataFrame\
.where("InvoiceDate < '2010-12-13'")
testDataFrame = preppedDataFrame\
.where("InvoiceDate >= '2010-12-13'")

6) Créer un StringIndexer qui permet de transformer les jours de semaine présents dans
la colonne day_of_week en valeurs numériques correspondantes.


In [13]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
.setInputCol("day_of_week")\
.setOutputCol("day_of_week_index")

7) En utilisant le StringIndexer, Spark peut par exemple représenter samedi par 6 et
lundi par 1. Cependant, avec ce schéma de numérotation, nous indiquons
implicitement que samedi est supérieur à lundi (par des valeurs numériques pures).
Ceci est évidemment incorrect. Comment résoudre ce problème ?


In [14]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
.setInputCol("day_of_week_index")\
.setOutputCol("day_of_week_encoded")

8) Créer un VectorAssembler contenant trois attributs : UnitPrice, Quantity, et
day_of_week_encoded.

In [15]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler()\
.setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
.setOutputCol("features")

9) Créer un pipeline configuré avec les résultats des étapes 6, 7 et 8.

In [16]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline()\
.setStages([indexer, encoder, vectorAssembler])

10) Notre StringIndexer doit savoir combien de valeurs uniques il y a à indexer, comment
résoudre ce problème ?

In [17]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

11) Transformer les données de l’ensemble d’apprentissage en se basant sur les étapes
(stages) du pipeline.

In [18]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

12)Créer une instance de KMeans, on supposse que le nombre de clusters est 20

In [19]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
.setK(20)

13) Lancer l’apprentissage de KMeans en se basant sur l’ensemble obtenu dans l’étape 11.

In [21]:
kmModel = kmeans.fit(transformedTraining)

14) Effectuer des prédictions sur l’ensemble de test.

In [22]:
transformedTest = fittedPipeline.transform(testDataFrame)
predictions = kmModel.transform(transformedTest)

15)Calculer le coefficient de silhouette.

In [23]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)